In [25]:
#!pip install python-slugify
# Python Script to convert WooCommerce Product Export CSV to Magento CSV
from slugify import slugify
import csv
import urllib.request

PATH_TO_MAGE_CSV = 'data/products/catalog_product.csv' #Magento Sample CSV
PATH_TO_WOO_CSV = 'data/products/wc-product-export.csv' #WooCommerce Products Export CSV
PATH_TO_OUTPUT_CSV = 'data/products/import_products.csv' #WooCommerce Products Export CSV

header_db = []
wc_db = []
dbDef = {}
wcDef = {}

with open(PATH_TO_MAGE_CSV, newline='', encoding='utf8') as f:
    reader = csv.reader(f)     
    for row in reader:
        header_db.append(row) 

#Add to missing columns to magento header (images)
header_db[0].append('base_image')
header_db[0].append('small_image')
header_db[0].append('thumbnail_image')
header_db[0].append('additional_images')

# Get magento headers and make a Dictionary of indices
list_size = len(header_db[0])
for x in range(0, list_size):
    dbDef[header_db[0][x]] = x

# now open Woocommerce CSV

with open(PATH_TO_WOO_CSV, newline='', encoding='utf8') as f:
    wcreader = csv.reader(f)     
    for row in wcreader:
        wc_db.append(row) 

# Get WooComerce headers and make a Dictionary of indices
wclist_size = len(wc_db[0])

for x in range(0, wclist_size):
    new_name = wc_db[0][x].replace(' ', '_')
    new_name = new_name.replace(':', '_')
    wcDef["wc_"+new_name] = x
    #globals()[f"wc_"+new_name] = x

#Prepare new list and write magento headers
write_list = []
write_list.append(header_db[0])

def appendCat(str): 
    if(str != None and str != ''):
        return '/' + str.replace(' > ', '/')
    return str

def formatImages(img,mode=0):
    imgList = []
    imgList = img.split(',',1)
    
    newlist =[]
    
    for image in imgList:
        
        tmp = []
        tmp = image.rsplit('/',1)
        newlist.append(tmp[1])
    if mode ==1:
        return newlist[0]
    else:
        str1 = ''
        sep = ','
        for i in range(1,len(newlist)):
            if str1 != '':
                str1 = str1 + sep     
        return str1
    
def cleanN(str):
    if(str != None and str != ''):
        str = str.replace('\r\\n', '<br>')
        str = str.replace('\t', '')
    return str

def getAllImages(images):
    imgList = []
    tmpname = ''
    imgList = images.split(',')
    for img in imgList:
        img = img.replace(' ', '')
        tmpname = img.rsplit('/',1)
        urllib.request.urlretrieve(img, tmpname[1])


for i in range(1,wclist_size):
    if wc_db[i][wcDef['wc_Type']] == 'simple' and wc_db[i][wcDef['wc_Regular_price']] !='' and wc_db[i][wcDef['wc_SKU']] !='':
        #Price and SKU are madatory
        new_list = []
        new_list = [''] * len(header_db[0])
        
        #Map Fields
        new_list[dbDef['sku']] = wc_db[i][wcDef['wc_SKU']]
        new_list[dbDef['attribute_set_code']] = 'Default'
        new_list[dbDef['product_type']] = wc_db[i][wcDef['wc_Type']]
        new_list[dbDef['categories']] = 'Default Category' + appendCat(wc_db[i][wcDef['wc_Categories']])
        new_list[dbDef['name']] = wc_db[i][wcDef['wc_Name']]
        new_list[dbDef['description']] = cleanN(wc_db[i][wcDef['wc_Description']])
        new_list[dbDef['short_description']]  = cleanN(wc_db[i][wcDef['wc_Short_description']])
        new_list[dbDef['product_websites']]  = 'base'
        new_list[dbDef['tax_class_name']]= 'Taxable Goods'
        new_list[dbDef['visibility']]= 'Catalog, Search'
        new_list[dbDef['price']]  = wc_db[i][wcDef['wc_Regular_price']]
        new_list[dbDef['url_key']] = slugify(wc_db[i][wcDef['wc_Name']])
        new_list[dbDef['meta_title']] = wc_db[i][wcDef['wc_Name']]
        #Map Images Field
        new_list[dbDef['base_image']] = formatImages(wc_db[i][wcDef['wc_Images']],1)
        new_list[dbDef['thumbnail_image']] = formatImages(wc_db[i][wcDef['wc_Images']],1)
        new_list[dbDef['small_image']] = formatImages(wc_db[i][wcDef['wc_Images']],1)
        new_list[dbDef['additional_images']] = formatImages(wc_db[i][wcDef['wc_Images']],0)
        write_list.append(new_list)
        
        #uncomment to get all and save images
        #getAllImages(wc_db[i][wc_Images])

with open(PATH_TO_OUTPUT_CSV, 'w', newline='', encoding='UTF8') as f:
    #create the csv writer
    writer = csv.writer(f)
    for row in write_list:
        writer.writerow(row)